In [30]:
from azureml.core import Workspace, Experiment

workspace_name = "ml-course"
experiment_name = "udacity-project-1-1"

compute_cluster_name = "worker"
vm_size = 'STANDARD_D2_V2'
max_nodes = 4

project_folder = './'
training_script_name = 'train.py'
outputs_folder_path = "outputs"
model_file_name = "model.pkl"

dataset_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
primary_metric_name = 'Accuracy'
hyperdrive_model_name = "hd_classifier"
automl_model_name = "automl_classifier"

ws = Workspace.get(name=workspace_name)
exp = Experiment(workspace=ws, name=experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: ml-course
Azure region: westeurope
Subscription id: f08c5f25-28be-4c21-993c-ad64d5c84d3a
Resource group: ML


In [31]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes = max_nodes)
cpu_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)


In [32]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.5, 0.99),
    "--max_iter": choice(1, 2, 4, 8, 16, 32, 64, 128)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1)

# Create a SKLearn estimator for use with train.py
est = SKLearn(
        source_directory=project_folder,
        entry_script=training_script_name,
        script_params={'--C':1.0, '--max_iter':2},
        compute_target=cpu_cluster
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                         hyperparameter_sampling=ps,
                                         policy=policy,
                                         primary_metric_name=primary_metric_name,
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=4,
                                         max_concurrent_runs=4)

In [33]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

In [35]:
hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status() == "Completed")

best_run = hyperdrive_run.get_best_run_by_primary_metric()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print(parameter_values)

model_parameters = dict(zip(parameter_values[::2], parameter_values[1::2]))
model_parameters['--save'] = True

from azureml.train.estimator import Estimator
model_est = Estimator(source_directory='.',
                      entry_script=training_script_name,
                      script_params=model_parameters,
                      compute_target=cpu_cluster,
                      conda_packages=['pandas==0.23.4',
                                      'scikit-learn==0.21.3',
                                      'lightgbm==2.2.1'])

model_run = exp.submit(model_est)
model_run.wait_for_completion(show_output=True)

RunId: HD_bea1d07f-83ac-4805-a9e5-559323e9b1ec
Web View: https://ml.azure.com/experiments/udacity-project-1-1/runs/HD_bea1d07f-83ac-4805-a9e5-559323e9b1ec?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/ml-course

Execution Summary
RunId: HD_bea1d07f-83ac-4805-a9e5-559323e9b1ec
Web View: https://ml.azure.com/experiments/udacity-project-1-1/runs/HD_bea1d07f-83ac-4805-a9e5-559323e9b1ec?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/ml-course

['--C', '0.792950901660784', '--max_iter', '64']
RunId: udacity-project-1-1_1610356331_02e4922e
Web View: https://ml.azure.com/experiments/udacity-project-1-1/runs/udacity-project-1-1_1610356331_02e4922e?wsid=/subscriptions/f08c5f25-28be-4c21-993c-ad64d5c84d3a/resourcegroups/ML/workspaces/ml-course

Streaming azureml-logs/55_azureml-execution-tvmps_14217070750dced54405645d9aa0169cc5eb4ed721a20b959b6a9a6230f3a8e5_d.txt

2021-01-11T09:16:51Z Starting output-watcher...
2021-01

{'runId': 'udacity-project-1-1_1610356331_02e4922e',
 'target': 'worker',
 'status': 'Completed',
 'startTimeUtc': '2021-01-11T09:16:52.45576Z',
 'endTimeUtc': '2021-01-11T09:20:36.293749Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '72108842-9cef-4a4b-89d7-69ba2c86d7c1',
  'azureml.git.repository_uri': 'https://github.com/udacity/nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files.git',
  'mlflow.source.git.repoURL': 'https://github.com/udacity/nd00333_AZMLND_Optimizing_a_Pipeline_in_Azure-Starter_Files.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': 'f5dbac53eab443d520913534cd1fdb4a9db6fe0d',
  'mlflow.source.git.commit': 'f5dbac53eab443d520913534cd1fdb4a9db6fe0d',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script':

In [36]:
model = model_run.register_model(model_name=hyperdrive_model_name, model_path=os.path.join(outputs_folder_path, model_file_name))
print(model.name, model.version, sep = '\t')

hd_classifier	1


In [38]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from train import clean_data

ds = TabularDatasetFactory.from_delimited_files(dataset_path)
x, y = clean_data(ds)

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric="accuracy",
    training_data=ds,
    n_cross_validations=5,
    compute_target=cpu_cluster,
    label_column_name='y')

# Submit your automl run
automl_run = exp.submit(automl_config)

# Retrieve and save your best automl model.
automl_run.wait_for_completion(show_output=True)
assert(automl_run.get_status() == "Completed")

automl_model = automl_run.register_model(model_name=automl_model_name, description="automl model")
print(automl_model.name, automl_model.version, sep = '\t')


Running on remote.

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+------

In [ ]:
cpu_cluster.delete()